In [ ]:
# Configuration - change these to match your environment
PROFILE = "main"  # Your connection profile from ~/.snowlib/connections.toml
SANDBOX_DATABASE = "O_CRI"  # Database where you can write test data
SANDBOX_SCHEMA = "PUBLIC"  # Schema where you can write test data

# Snowflake sample data (read-only, available to all accounts)
SAMPLE_DATABASE = "SNOWFLAKE_SAMPLE_DATA"
SAMPLE_SCHEMA = "TPCH_SF1"

## 1. Connection

snowlib uses `SnowflakeContext` to manage connections. It loads credentials from your profile and provides a reusable connection

In [ ]:
from snowlib import SnowflakeContext, list_profiles

# List available profiles
print("Available profiles:", list_profiles())

In [ ]:
# Create a context (this establishes the connection)
ctx = SnowflakeContext(profile=PROFILE)

In [ ]:
print(f"Connected to: {ctx.current_account}")
print(f"User: {ctx.current_user}")
print(f"Warehouse: {ctx.current_warehouse}")
print(f"Role: {ctx.current_role}")

## 1b. Session (Context-Bound Convenience)

In [ ]:
from snowlib import create_session

# Create a session - no need to pass context to every call
session = create_session(profile=PROFILE)

In [ ]:
# Query directly - no ctx parameter needed
df = session.query(f"SELECT * FROM {SAMPLE_DATABASE}.{SAMPLE_SCHEMA}.REGION")
df

In [ ]:
# Models are also bound - create objects with factory methods
table = session.table(SAMPLE_DATABASE, SAMPLE_SCHEMA, "CUSTOMER")
df = table.read(limit=5)
df

## 2. Execution Layer

The primitives layer provides functions for executing SQL directly

In [ ]:
from snowlib import execute_sql, query

# execute_sql returns a QueryResult with metadata
result = execute_sql("SELECT CURRENT_VERSION() AS version", ctx)
print(f"Query ID: {result.query_id}")
print(f"Row count: {result.rowcount}")

In [ ]:
# query() returns a DataFrame directly - great for quick data exploration
df = query(f"SELECT * FROM {SAMPLE_DATABASE}.{SAMPLE_SCHEMA}.NATION LIMIT 10", ctx)
df

In [ ]:
# Query with aggregation
df = query(f"""
    SELECT 
        n.n_name AS nation,
        COUNT(*) AS customer_count
    FROM {SAMPLE_DATABASE}.{SAMPLE_SCHEMA}.CUSTOMER c
    JOIN {SAMPLE_DATABASE}.{SAMPLE_SCHEMA}.NATION n ON c.c_nationkey = n.n_nationkey
    GROUP BY n.n_name
    ORDER BY customer_count DESC
    LIMIT 10
""", ctx)
df

## 3. Models Layer

The models layer provides an object-oriented interface

In [ ]:
from snowlib import Database, Schema, Table

# Create a Database object
db = Database(SAMPLE_DATABASE, ctx)

print(f"Database: {db.name}")
print(f"Exists: {db.exists()}")

In [ ]:
# List schemas in the database
schemas = db.schemas
print(f"Found {len(schemas)} schemas:")
for s in schemas:
    print(f"  - {s.name}")

In [ ]:
# Navigate to a schema and list tables
schema = db.schema(SAMPLE_SCHEMA)
tables = schema.tables

print(f"Tables in {schema.fqn}:")
for t in tables:
    print(f"  - {t.name}")

In [ ]:
# Work with a specific table
nation_table = schema.table("NATION")

print(f"Table: {nation_table.fqn}")
print(f"Exists: {nation_table.exists()}")

In [ ]:
# Get column information
columns = nation_table.columns
print(f"Columns in {nation_table.name}:")
for col in columns:
    print(f"  - {col.name}")

In [ ]:
# Read data from a table
df = nation_table.read(limit=5)
df

In [ ]:
# Describe table structure
nation_table.describe()

### Chained Navigation

Models support chaining:

In [ ]:
# One-liner: Database -> Schema -> Table -> Read
df = Database(SAMPLE_DATABASE, ctx).schema(SAMPLE_SCHEMA).table("REGION").read()
df

## 4. Writing Data

- `Table.write()`
- `Table.truncate()`
- `Table.drop()`

In [ ]:
import pandas as pd

# Create a test table in your sandbox
sandbox = Database(SANDBOX_DATABASE, ctx).schema(SANDBOX_SCHEMA)
test_table = sandbox.table("SNOWLIB_DEMO_TEST")

# Create sample data
sample_data = pd.DataFrame({
    "ID": [1, 2, 3],
    "NAME": ["Alice", "Bob", "Charlie"],
    "VALUE": [100, 200, 300]
})

# Write data (creates table if it doesn't exist)
test_table.write(sample_data, if_exists="replace")
print(f"Wrote {len(sample_data)} rows to {test_table.fqn}")

In [ ]:
# Read it back
test_table.read()

In [ ]:
# Append more data
more_data = pd.DataFrame({
    "ID": [4, 5],
    "NAME": ["Dave", "Eve"],
    "VALUE": [400, 500]
})
test_table.write(more_data, if_exists="append")

print(f"Now have {len(test_table.read())} rows")
test_table.read()

In [ ]:
# Get table metadata
metadata = test_table.metadata
print(f"Rows: {metadata.get('rows')}")
print(f"Created: {metadata.get('created_on')}")
print(f"Owner: {metadata.get('owner')}")

In [ ]:
# Clean up
test_table.drop()
print(f"Dropped {test_table.fqn}")

## 5. SQLAlchemy Integration

In [ ]:
from snowlib.sqlalchemy import create_engine_from_profile
from sqlalchemy import text

# Create engine from profile
engine = create_engine_from_profile(PROFILE)

In [ ]:
# Use standard SQLAlchemy patterns
with engine.connect() as conn:
    result = conn.execute(text(f"""
        SELECT n_name, n_regionkey 
        FROM {SAMPLE_DATABASE}.{SAMPLE_SCHEMA}.NATION 
        WHERE n_regionkey = :region
    """), {"region": 1})
    
    for row in result:
        print(row)

In [ ]:
# Or use pandas read_sql for DataFrame results
import pandas as pd

df = pd.read_sql(
    f"SELECT * FROM {SAMPLE_DATABASE}.{SAMPLE_SCHEMA}.REGION",
    engine
)
df

## 6. Cleanup

In [ ]:
ctx.close()
print("Connection closed")